# Quick EDA - Christmas Tree Packing

In [1]:
import pandas as pd
import numpy as np

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')
df.head(10)

Shape: (20100, 4)
Columns: ['id', 'x', 'y', 'deg']


,id,x,y,deg
0,001_0,s0.0,s0.0,s90.0
1,002_0,s0.0,s0.0,s90.0
2,002_1,s0.202736,s-0.511271,s90.0
3,003_0,s0.0,s0.0,s90.0
4,003_1,s0.202736,s-0.511271,s90.0
5,003_2,s0.5206,s0.177413,s180.0
6,004_0,s0.0,s0.0,s90.0
7,004_1,s0.202736,s-0.511271,s90.0
8,004_2,s0.5206,s0.177413,s180.0
9,004_3,s-0.818657,s-0.228694,s180.0


In [2]:
# Analyze the structure - how many trees per configuration
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
df['tree_idx'] = df['id'].apply(lambda x: int(x.split('_')[1]))

print(f"Number of configurations: {df['n'].nunique()}")
print(f"Configuration range: {df['n'].min()} to {df['n'].max()}")
print(f"Total trees: {len(df)}")
print(f"Expected total: {sum(range(1, 201))} = 1+2+...+200")

# Verify counts per configuration
counts = df.groupby('n').size()
print(f"\nFirst 10 configurations have trees: {counts.head(10).tolist()}")
print(f"Last 5 configurations have trees: {counts.tail(5).tolist()}")

Number of configurations: 200
Configuration range: 1 to 200
Total trees: 20100
Expected total: 20100 = 1+2+...+200

First 10 configurations have trees: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Last 5 configurations have trees: [196, 197, 198, 199, 200]
